# Methodology

This chapter describes our analytical approach for estimating the budgetary impacts of Social Security benefit taxation reforms. We employ microsimulation modeling using PolicyEngine, an open-source platform that provides detailed estimates of policy changes on federal tax revenue and household-level impacts.

## Microsimulation Framework

### PolicyEngine Platform

Our analysis uses PolicyEngine US, an open-source tax-benefit microsimulation model. PolicyEngine provides several advantages for this analysis:

- **Comprehensive Tax Code Implementation**: Complete modeling of federal income tax provisions, including Social Security benefit taxation rules
- **Current Data**: Based on the most recent enhanced Current Population Survey data (2024)
- **Policy Flexibility**: Allows detailed specification of complex policy reforms
- **Transparency**: Open-source codebase enables verification and replication
- **Validation**: Regular benchmarking against official government estimates

In [ ]:
# Import necessary libraries and set up environment
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import numpy as np
from IPython.display import display, Markdown, HTML
import plotly.express as px
import plotly.graph_objects as go
import os
import warnings

# Define PolicyEngine's color palette
BLACK = "#000000"
BLUE_LIGHT = "#D8E6F3"
BLUE_PRIMARY = "#2C6496"
DARK_GRAY = "#616161"
LIGHT_GRAY = "#F2F2F2"
WHITE = "#FFFFFF"

# Suppress warnings for clean output
warnings.filterwarnings('ignore')

print("PolicyEngine libraries loaded successfully!")

## Policy Implementation

### Reform Specification

Each policy option is implemented through PolicyEngine's reform system, which allows precise specification of tax rate changes, threshold modifications, deduction changes, credit implementation, and phase-in schedules.

Below are the reform functions for each policy option:

In [ ]:
# Define reform functions for all policy options

def get_option1_reform():
    """Option 1: Full Repeal of Social Security Benefits Taxation"""
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.additional": {
            "2026-01-01.2100-12-31": 0
        }
    }, country_id="us")

def get_option2_reform():
    """Option 2: Taxation of 85% of Social Security Benefits"""
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0.85
        },
        "gov.irs.social_security.taxability.threshold.base.main.JOINT": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SINGLE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SEPARATE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SURVIVING_SPOUSE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2100-12-31": 0
        }
    }, country_id="us")

def get_option3_reform():
    """Option 3: 85% Taxation with Permanent Senior Deduction Extension"""
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0.85
        },
        "gov.irs.social_security.taxability.threshold.base.main.JOINT": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SINGLE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SEPARATE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SURVIVING_SPOUSE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.contrib.crfb.senior_deduction_extension.applies": {
            "2026-01-01.2100-12-31": True
        }
    }, country_id="us")

def get_option4_reform():
    """Option 4: Social Security Tax Credit System ($500 Credit)"""
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0.85
        },
        "gov.irs.social_security.taxability.threshold.base.main.JOINT": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SINGLE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SEPARATE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SURVIVING_SPOUSE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.contrib.crfb.ss_credit.in_effect": {
            "2026-01-01.2100-12-31": True
        },
        "gov.contrib.crfb.ss_credit.amount.JOINT": {
            "2026-01-01.2100-12-31": 500
        },
        "gov.contrib.crfb.ss_credit.amount.SINGLE": {
            "2026-01-01.2100-12-31": 500
        },
        "gov.contrib.crfb.ss_credit.amount.SEPARATE": {
            "2026-01-01.2100-12-31": 500
        },
        "gov.contrib.crfb.ss_credit.amount.SURVIVING_SPOUSE": {
            "2026-01-01.2100-12-31": 500
        },
        "gov.contrib.crfb.ss_credit.amount.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2100-12-31": 500
        },
        "gov.irs.deductions.senior_deduction.amount": {
            "2026-01-01.2100-12-31": 0
        }
    }, country_id="us")

def get_option5_reform():
    """Option 5: Roth-Style Swap"""
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.additional": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.contrib.crfb.tax_employer_payroll_tax.in_effect": {
            "2026-01-01.2100-12-31": True
        },
        "gov.contrib.crfb.tax_employer_payroll_tax.percentage": {
            "2026-01-01.2100-12-31": 1.0
        }
    }, country_id="us")

def get_option6_reform():
    """Option 6: Phased Roth-Style Swap"""
    reform_dict = {
        "gov.contrib.crfb.tax_employer_payroll_tax.in_effect": {
            "2026-01-01.2100-12-31": True
        },
        "gov.contrib.crfb.tax_employer_payroll_tax.percentage": {
            "2026-01-01.2026-12-31": 0.1307,
            "2027-01-01.2027-12-31": 0.2614,
            "2028-01-01.2028-12-31": 0.3922,
            "2029-01-01.2029-12-31": 0.5229,
            "2030-01-01.2030-12-31": 0.6536,
            "2031-01-01.2031-12-31": 0.7843,
            "2032-01-01.2032-12-31": 0.9150,
            "2033-01-01.2100-12-31": 1.0
        },
        "gov.irs.social_security.taxability.rate.base": {
            "2029-01-01.2029-12-31": 0.45,
            "2030-01-01.2030-12-31": 0.40,
            "2031-01-01.2031-12-31": 0.35,
            "2032-01-01.2032-12-31": 0.30,
            "2033-01-01.2033-12-31": 0.25,
            "2034-01-01.2034-12-31": 0.20,
            "2035-01-01.2035-12-31": 0.15,
            "2036-01-01.2036-12-31": 0.10,
            "2037-01-01.2037-12-31": 0.05,
            "2038-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.additional": {
            "2029-01-01.2029-12-31": 0.80,
            "2030-01-01.2030-12-31": 0.75,
            "2031-01-01.2031-12-31": 0.70,
            "2032-01-01.2032-12-31": 0.65,
            "2033-01-01.2033-12-31": 0.60,
            "2034-01-01.2034-12-31": 0.55,
            "2035-01-01.2035-12-31": 0.50,
            "2036-01-01.2036-12-31": 0.45,
            "2037-01-01.2037-12-31": 0.40,
            "2038-01-01.2038-12-31": 0.35,
            "2039-01-01.2039-12-31": 0.30,
            "2040-01-01.2040-12-31": 0.25,
            "2041-01-01.2041-12-31": 0.20,
            "2042-01-01.2042-12-31": 0.15,
            "2043-01-01.2043-12-31": 0.10,
            "2044-01-01.2044-12-31": 0.05,
            "2045-01-01.2100-12-31": 0
        }
    }
    return Reform.from_dict(reform_dict, country_id="us")

print("Policy reform functions defined successfully!")

## Running Calculations and Generating Results

The following section calculates the budgetary impacts for each policy option using PolicyEngine microsimulation and exports the results to the policy_impacts.md file.

In [ ]:
def calculate_yearly_impact(reform, year, baseline_income_tax):
    """
    Calculate the budgetary impact for a given reform and year.
    Uses pre-computed baseline to avoid redundant calculations.
    Returns impact in billions (negative = cost, positive = savings)
    """
    try:
        if reform is not None:
            reformed = Microsimulation(reform=reform, dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")
            reformed_income_tax = reformed.calculate("income_tax", map_to="household", period=year)
            difference_income_tax = baseline_income_tax - reformed_income_tax
            impact_billions = difference_income_tax.sum() / 1e9
        else:
            # Return placeholder value if reform not implemented
            impact_billions = 0.0
            
        return round(impact_billions, 1)
    except Exception as e:
        print(f"Error calculating impact for year {year}: {e}")
        return 0.0

print("Calculation functions defined successfully!")

In [ ]:
# Pre-compute baselines for all years to avoid redundant calculations
print("Pre-computing baselines for all years...")
baselines = {}
years = list(range(2026, 2036))

for year in years:
    print(f"  Computing baseline for {year}...")
    baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")
    baseline_income_tax = baseline.calculate("income_tax", map_to="household", period=year)
    baselines[year] = baseline_income_tax
    
print("Baseline computation complete!\n")

In [ ]:
# Calculate impacts for each policy option
print("Calculating policy impacts...")

# Define the 6 policy options we want to analyze
policy_options = {
    'Option 1': get_option1_reform(),
    'Option 2': get_option2_reform(), 
    'Option 3': get_option3_reform(),
    'Option 4': get_option4_reform(),
    'Option 5': get_option5_reform(),
    'Option 6': get_option6_reform()
}

# Calculate impacts for each option and year
yearly_data = {'Year': years}

for option_name, reform in policy_options.items():
    print(f"\nProcessing {option_name}...")
    impacts = []
    
    for year in years:
        print(f"  Year {year}: Computing...")
        impact = calculate_yearly_impact(reform, year, baselines[year])
        impacts.append(impact)
    
    yearly_data[option_name] = impacts

# Create DataFrame
yearly_df = pd.DataFrame(yearly_data)

# Calculate 10-year totals
totals = {}
for col in yearly_df.columns:
    if col != 'Year':
        totals[col] = yearly_df[col].sum()

print("\nCalculation complete!")
print("\n=== 10-Year Totals (2026-2035) ===")
for option, total in totals.items():
    print(f"{option}: {total:.1f} billion")

In [8]:
# Display the year-by-year table with totals
yearly_table_data = []
for i, year in enumerate(yearly_data['Year']):
    row = {'Year': year}
    for option in ['Option 1', 'Option 2', 'Option 3', 'Option 4', 'Option 5', 'Option 6']:
        row[option] = f"{yearly_data[option][i]:.1f}"
    yearly_table_data.append(row)

# Add totals row
totals_row_data = {'Year': '**2026-2035 Total**'}
for option in ['Option 1', 'Option 2', 'Option 3', 'Option 4', 'Option 5', 'Option 6']:
    totals_row_data[option] = f"**{totals[option]:.1f}**"
yearly_table_data.append(totals_row_data)

yearly_table_df = pd.DataFrame(yearly_table_data)

print("Year-by-Year Federal Budgetary Impact (Billions of Dollars)")
display(yearly_table_df)

Year-by-Year Federal Budgetary Impact (Billions of Dollars)


,Year,Option 1,Option 2,Option 3,Option 4,Option 5,Option 6
0,2026,86.3,-18.3,-18.3,-24.4,-58.2,-18.7
1,2027,91.8,-18.3,-18.3,-24.6,-61.4,-39.9
2,2028,96.9,-18.1,-18.1,-24.6,-65.6,-63.7
3,2029,105.9,-23.8,3.5,-2.9,-67.6,-83.3
4,2030,112.7,-24.0,3.5,-2.8,-73.5,-106.7
5,2031,118.4,-23.9,3.6,-2.4,-77.8,-130.7
6,2032,124.5,-23.7,3.7,-2.0,-82.0,-156.9
7,2033,130.7,-23.3,3.9,-1.5,-86.1,-174.9
8,2034,137.5,-23.1,4.2,-1.0,-90.1,-174.8
9,2035,144.3,-22.8,4.4,-0.6,-93.9,-173.9


In [9]:
# Function to write results to policy_impacts.md
def write_results_to_markdown():
    """
    Write the calculated results to policy_impacts.md file
    """
    
    # Prepare the summary table
    summary_data = [
        {"Policy Option": "**Option 1**: Full Repeal", "10-Year Impact (Billions)": f"${totals['Option 1']:.1f}"},
        {"Policy Option": "**Option 2**: 85% Taxation", "10-Year Impact (Billions)": f"${totals['Option 2']:.1f}"},
        {"Policy Option": "**Option 3**: 85% with Senior Deduction", "10-Year Impact (Billions)": f"${totals['Option 3']:.1f}"},
        {"Policy Option": "**Option 4**: Tax Credit ($500)", "10-Year Impact (Billions)": f"${totals['Option 4']:.1f}"},
        {"Policy Option": "**Option 5**: Roth-Style Swap", "10-Year Impact (Billions)": f"${totals['Option 5']:.1f}"},
        {"Policy Option": "**Option 6**: Phased Roth-Style", "10-Year Impact (Billions)": f"${totals['Option 6']:.1f}"}
    ]
    
    # Create markdown tables manually to avoid tabulate dependency issues
    
    # Summary table
    summary_table = "| Policy Option | 10-Year Impact (Billions) |\n|---|---|\n"
    for item in summary_data:
        summary_table += f"| {item['Policy Option']} | {item['10-Year Impact (Billions)']} |\n"
    
    # Year-by-year table with totals
    yearly_table = "| Year | Option 1 | Option 2 | Option 3 | Option 4 | Option 5 | Option 6 |\n"
    yearly_table += "|------|----------|----------|----------|----------|----------|----------|\n"
    
    # Add yearly data
    for i, year in enumerate(yearly_data['Year']):
        yearly_table += f"| {year} | ${yearly_data['Option 1'][i]:.1f} | ${yearly_data['Option 2'][i]:.1f} | ${yearly_data['Option 3'][i]:.1f} | ${yearly_data['Option 4'][i]:.1f} | ${yearly_data['Option 5'][i]:.1f} | ${yearly_data['Option 6'][i]:.1f} |\n"
    
    # Add totals row
    yearly_table += f"| **2026-2035 Total** | **${totals['Option 1']:.1f}** | **${totals['Option 2']:.1f}** | **${totals['Option 3']:.1f}** | **${totals['Option 4']:.1f}** | **${totals['Option 5']:.1f}** | **${totals['Option 6']:.1f}** |\n"
    
    # Determine if values are positive or negative for proper description
    def format_impact(value):
        if value < 0:
            return f"-${abs(value):.1f} billion"
        else:
            return f"${value:.1f} billion"
    
    # Write the complete markdown file
    content = f"""# Policy Impacts

This chapter presents the quantitative results of the analysis of six Social Security benefit taxation reform options. All estimates represent changes in federal tax revenue over the 10-year period 2026-2035, measured in billions of dollars. Negative values indicate revenue decreases, while positive values indicate revenue increases.

## Summary of 10-Year Budgetary Impacts

The microsimulation analysis shows the following budgetary impacts across the six policy options:

{summary_table}

## Year-by-Year Federal Budgetary Impact (2026-2035)

**Federal Budgetary Impact by Year (Billions of Dollars)**

{yearly_table}

## Detailed Results by Policy Option

### Option 1: Full Repeal of Social Security Benefits Taxation

**10-Year Impact: {format_impact(totals['Option 1'])}**

This option eliminates all federal income taxation of Social Security benefits. The revenue impact changes over time due to:
- Changes in the number of Social Security beneficiaries
- Changes in benefit levels
- Expiration of senior deduction in 2029

### Option 2: Taxation of 85% of Social Security Benefits

**10-Year Impact: {format_impact(totals['Option 2'])}**

This option taxes 85% of Social Security benefits for all recipients. The results show that expanding taxation to all beneficiaries at 85% does not offset the revenue change from eliminating income thresholds under the current senior deduction structure.

### Option 3: 85% Taxation with Permanent Senior Deduction Extension

**10-Year Impact: {format_impact(totals['Option 3'])}**

This option taxes 85% of Social Security benefits while permanently extending the senior deduction. The change after 2028 reflects the interaction between expanded benefit taxation and the permanent senior deduction.

### Option 4: Social Security Tax Credit System ($500 Credit)

**10-Year Impact: {format_impact(totals['Option 4'])}**

This option implements a $500 tax credit while expanding the taxation base to 85% of all benefits. The change after 2028 reflects the interaction between the credit system and the expiration of the senior deduction.

### Option 5: Roth-Style Swap

**10-Year Impact: {format_impact(totals['Option 5'])}**

This option replaces benefit taxation with employer payroll contribution taxation. The revenue impact changes over time as the difference between benefit taxation revenue and employer contribution taxation revenue evolves.

### Option 6: Phased Roth-Style Swap

**10-Year Impact: {format_impact(totals['Option 6'])}**

This option implements a phased transition that includes:
- Gradual reduction of benefit taxation revenue
- Gradual increase of employer contribution taxation
- Extended transition period

## Key Findings

### Revenue Impact Ranking

From smallest to largest absolute revenue impact:
1. **Option 3**: {format_impact(totals['Option 3'])}
2. **Option 4**: {format_impact(totals['Option 4'])}
3. **Option 2**: {format_impact(totals['Option 2'])}
4. **Option 5**: {format_impact(totals['Option 5'])}
5. **Option 1**: {format_impact(totals['Option 1'])}
6. **Option 6**: {format_impact(totals['Option 6'])}

### Impact of Senior Deduction Expiration

The scheduled 2028 expiration of the senior deduction affects the revenue patterns of several options:

- **Options 1, 5, 6**: Revenue impacts change after 2028
- **Option 2**: Revenue pattern changes after 2028
- **Option 3**: Different pattern due to permanent extension
- **Option 4**: Credit interacts with deduction expiration

### Policy Design Observations

1. **Tax Credit**: The $500 tax credit option combines a credit with expanded benefit taxation.

2. **Roth-Style Swap**: The analysis shows different revenue impacts between benefit taxation and employer payroll contribution taxation.

3. **Phase-in Effects**: Gradual implementation (Option 6) produces different revenue patterns than immediate implementation (Option 5).

4. **Senior Deduction**: The permanent senior deduction extension (Option 3) affects the net revenue impact of expanded benefit taxation.

## Distributional Considerations

The analysis focuses on aggregate budgetary impacts. The distributional effects vary by option:

- **Option 1**: Affects all current Social Security recipients
- **Options 2 & 3**: Changes tax treatment for beneficiaries currently below thresholds
- **Option 4**: Implements a credit for Social Security recipients
- **Options 5 & 6**: Changes the tax treatment between retirees and current workers

## Methodology Notes

These estimates are based on static microsimulation analysis and do not account for:
- Behavioral responses to tax changes
- Economic growth effects
- Administrative implementation costs
- State tax interactions

The results are presented for comparison with estimates from other analytical sources in the next chapter."""
    
    # Write to file
    with open('policy_impacts.md', 'w') as f:
        f.write(content)
    
    print("Results successfully written to policy_impacts.md")
    return content

# Execute the function to write results
markdown_content = write_results_to_markdown()

Results successfully written to policy_impacts.md


## Calculation Methodology

### Annual Impact Estimation

For each policy option and year (2026-2035), we calculate:

1. **Baseline Revenue**: Federal income tax revenue under current law
2. **Reform Revenue**: Federal income tax revenue under the policy reform
3. **Budgetary Impact**: Difference between baseline and reform revenue

The impact is calculated as: `Impact = Baseline Revenue - Reform Revenue`

- **Negative values** indicate costs (revenue loss)
- **Positive values** indicate savings (revenue gain)

## Data Foundation

### Enhanced Current Population Survey

The microsimulation is based on the Enhanced Current Population Survey (CPS) for 2024:

- **Sample Size**: Approximately 160,000 individuals in 60,000 households
- **Representative Coverage**: Nationally representative sample weighted to match U.S. population
- **Income Sources**: Detailed information on wages, Social Security benefits, pensions, and other income
- **Demographics**: Age, filing status, state of residence, and other relevant characteristics
- **Enhancement**: Survey data enhanced with statistical matching to capture income and benefit details

### Tax Calculator Engine

PolicyEngine's tax calculator implements current law provisions including:

- Federal income tax brackets and rates
- Standard and itemized deductions
- Social Security benefit taxation rules (current two-tier system)
- Senior deduction provisions
- Credits and other tax preferences
- Alternative Minimum Tax calculations

## Technical Specifications

### Software Environment

- **PolicyEngine Version**: Latest stable release (2024)
- **Python Version**: 3.10+
- **Key Dependencies**: NumPy, Pandas for data processing
- **Computational Resources**: Analysis performed on standard computing environment

### Data Processing

The analysis workflow includes:

1. **Baseline Calculation**: Pre-compute baseline tax liabilities for all years
2. **Reform Implementation**: Apply policy changes through PolicyEngine reform system
3. **Impact Calculation**: Calculate difference between reform and baseline scenarios
4. **Quality Assurance**: Validate results against known benchmarks
5. **Output Generation**: Format results for analysis and presentation

## Limitations and Assumptions

### Static Analysis

Our analysis uses static microsimulation, meaning:
- No behavioral responses to tax changes are modeled
- Labor supply, retirement timing, and benefit claiming decisions held constant
- Savings and investment responses not captured
- Economic growth effects not included

### Data Limitations

The Enhanced CPS data has several limitations:
- Survey data may underrepresent very high-income households
- Some income sources may be underreported
- Administrative data matching not available for all variables
- Sample size limits precision for small subgroups

### Policy Implementation

Our policy implementations make several assumptions:
- Full compliance with new tax provisions
- No administrative costs or implementation delays
- Perfect information and understanding by taxpayers
- No interaction with state tax systems

### Economic Assumptions

The analysis assumes:
- Current law economic assumptions (inflation, wage growth)
- No macroeconomic feedback effects from policy changes
- Stable demographic and benefit program parameters
- No changes to Social Security benefit levels or eligibility